# Comparison of track parameter in full and fast simulation

I want to plot the 5 track parameters and the 15 covariance parameters for $H \rightarrow u \bar{u}$ for the leading tracks in fast and full simulation. Therefore, I need to make sure to plot the same track params and especially find out what the 15 covariance parameters mean.